In [491]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
if (code_show){
$('div.input').hide();
} else {
$('div.input').show();
}
code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="View/Hide Raw Code"></form>''')

# Converting the GenBank File to SBOL
1. Upload the GenBank file or place in the same folder.
2. Enter the file name and the URI prefix that you want to use.
3. Enter a version (optional).
4. Hit send and it will create an SBOL file with the same name.

In [492]:
from sbol import *
import ipywidgets as widgets
from IPython.display import Javascript, display, clear_output
import requests
import os
import os.path

fileName = widgets.Text(
    value = '',
    placeholder = 'Enter file name',
    description = 'File Name:',
    disabled = False
)

URIPrefix = widgets.Text(
    value = '',
    placeholder = 'Enter URI Prefix',
    description = 'URI Prefix:',
    disabled = False
)

version = widgets.Text(
    value = '',
    placeholder = 'Enter Version (optional)',
    description = 'Version:',
    disabled = False
)

sendButton = widgets.Button(
    description = 'Send',
    disabled = False,
    button_style = '',
    tooltip = 'Converts Genbank file to SBOL',
    icon = ''
)

out = widgets.Output()

display(fileName)
display(URIPrefix)
display(version)
display(sendButton)

@out.capture()
def sendRequest(sender):
    clear_output()
    if fileName.value == '' and URIPrefix.value == '':
        print('You need both a file name and URI Prefix!')
        return
    if fileName.value == '':
        print('You need a file name!')
        return
    if URIPrefix.value == '':
        print('You need a URI Prefix!')
        return
    setHomespace(URIPrefix.value)
    gbFile = open(fileName.value).read()

    request = { 'options': {'language' : 'SBOL2',
                        'test_equality': False,
                        'check_uri_compliance': False,
                        'check_completeness': False,
                        'check_best_practices': False,
                        'fail_on_first_error': False,
                        'provide_detailed_stack_trace': False,
                        'subset_uri': '',
                        'uri_prefix': URIPrefix.value,
                        'version': '',
                        'insert_type': False,
                        'main_file_name': 'main file',
                        'diff_file_name': 'comparison file',
                                },
            'return_file': True,
            'main_file': gbFile
          }

    response = requests.post("http://www.async.ece.utah.edu/validate/", json=request)
    xmlResponse = response.json()['result']
    
    sbolFile = open(fileName.value[:-3] + '.xml', 'w+')
    sbolFile.write(xmlResponse)
    print('Success!')

sendButton.on_click(sendRequest)
display(out)
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))

Text(value='', description='File Name:', placeholder='Enter file name')

Text(value='', description='URI Prefix:', placeholder='Enter URI Prefix')

Text(value='', description='Version:', placeholder='Enter Version (optional)')

Button(description='Send', style=ButtonStyle(), tooltip='Converts Genbank file to SBOL')

Output()

<IPython.core.display.Javascript object>

# Reading in the SBOL File
Reads in an SBOL file and displays the Component Definitions.

In [493]:
from sbol import *
import ipywidgets as widgets
from IPython.display import Javascript, display, clear_output
import requests
import os
import os.path

fileNameSBOL = widgets.Text(
    value = '',
    placeholder = 'Enter SBOL file name',
    description = 'File Name:',
    disabled = False
)

submit = widgets.Button(
    description = 'Submit',
    disabled = False,
    button_style = '',
    tooltip = 'Reads in the SBOL file',
    icon = ''
)

out = widgets.Output()

display(fileNameSBOL)
display(submit)

doc = Document()
setHomespace('https://bu.edu/ben')
Config.setOption('sbol_typed_uris', False)
PATH = './'

originalCDs = []


@out.capture()
def showDoc(sender):
    global originalCDs
    clear_output()
    if fileNameSBOL.value != '':
        if os.path.isfile(PATH + fileNameSBOL.value) and (fileNameSBOL.value[-3:] == 'xml' or fileNameSBOL.value[-3:] == 'rdf'):
            doc.read(fileNameSBOL.value)
            originalCDs = doc.componentDefinitions
            print(doc)
        else:
            print('Invalid file name!')
    else:
        print('You need a file name!')
    
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))


submit.on_click(showDoc)
display(out)

Text(value='', description='File Name:', placeholder='Enter SBOL file name')

Button(description='Submit', style=ButtonStyle(), tooltip='Reads in the SBOL file')

Output()

# Add a Component

In [501]:
from sbol import *
import ipywidgets as widgets
from IPython.display import clear_output, Markdown

componentTypes = ['DNA', 'RNA', 'Protein', 'Small Molecule', 'Complex']

componentRoles = ['', 'Miscellaneous', 'Promoter', 'RBS', 'CDS', 'Terminator',
                  'Gene', 'Operator', 'Engineered Gene', 'mRNA', 'Effector']

typesMap = {'DNA':'http://www.biopax.org/release/biopax-level3.owl#DnaRegion',
            'RNA':'http://www.biopax.org/release/biopax-level3.owl#RnaRegion',
            'Protein':'http://www.biopax.org/release/biopax-level3.owl#Protein',
            'Small Molecule':'http://www.biopax.org/release/biopax-level3.owl#SmallMolecule',
            'Complex':'http://www.biopax.org/release/biopax-level3.owl#Complex'}

rolesMap = {'Miscellaneous':'http://identifiers.org/so/SO:0000001',
            'Promoter':'http://identifiers.org/so/SO:0000167',
            'RBS':'http://identifiers.org/so/SO:0000139',
            'CDS':'http://identifiers.org/so/SO:0000316',
            'Terminator':'http://identifiers.org/so/SO:0000141',
            'Gene':'http://identifiers.org/so/SO:0000704',
            'Operator':'http://identifiers.org/so/SO:0000057',
            'Engineered Gene':'http://identifiers.org/so/SO:0000280',
            'mRNA':'http://identifiers.org/so/SO:0000234',
            'Effector':'http://identifiers.org/chebi/CHEBI:35224',
            'Transcription Factor':'http://identifiers.org/go/GO:0003700'}

componentName = widgets.Text(
    value = '',
    placeholder = 'Enter a component name',
    description = 'Name:',
    disabled = False,
)

typeDrop = widgets.Dropdown(
    options = componentTypes,
    description = 'Type:',
    disabled = False
)

roleDrop = widgets.Dropdown(
    options = componentRoles,
    description = 'Role:',
    disabled = False
)

createButton = widgets.Button(
    description = 'Create',
    disabled = False,
    button_style = '',
    tooltip = 'Creates a new Component Definition ',
    icon = ''
)

outBox = widgets.Output()

display(componentName)
display(typeDrop)
display(roleDrop)
display(createButton)
display(outBox)

@outBox.capture()
def createButtonHandler(sender):
    clear_output()
    if componentName.value == '':
        print('You need a component name!')
        return
    
    try:
        newComponentDefinition = ComponentDefinition(componentName.value, typesMap[typeDrop.value])
        if roleDrop.value != '':
            newComponentDefinition.roles = rolesMap[roleDrop.value]
    except Exception as e:
        print(e)
        return
    
    try:
        doc.addComponentDefinition(newComponentDefinition)
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
        print('Component created!')
    except RuntimeError:
        print('Component name already exists!')

createButton.on_click(createButtonHandler)

Text(value='', description='Name:', placeholder='Enter a component name')

Dropdown(description='Type:', options=('DNA', 'RNA', 'Protein', 'Small Molecule', 'Complex'), value='DNA')

Dropdown(description='Role:', options=('', 'Miscellaneous', 'Promoter', 'RBS', 'CDS', 'Terminator', 'Gene', 'O…

Button(description='Create', style=ButtonStyle(), tooltip='Creates a new Component Definition ')

Output()

# Component Definitions

In [508]:
from sbol import *
import ipywidgets as widgets
from IPython.display import clear_output, Markdown

typeDict = {'http://www.biopax.org/release/biopax-level3.owl#DnaRegion':'DNA',
            'http://www.biopax.org/release/biopax-level3.owl#RnaRegion':'RNA',
            'http://www.biopax.org/release/biopax-level3.owl#Protein':'Protein',
            'http://www.biopax.org/release/biopax-level3.owl#SmallMolecule': 'Small Molecule',
            'http://www.biopax.org/release/biopax-level3.owl#Complex':'Complex'}

roleDict = {'http://identifiers.org/so/SO:0000001':'Miscellaneous', 
            'http://identifiers.org/so/SO:0000167':'Promoter',
            'http://identifiers.org/so/SO:0000139':'RBS',
            'http://identifiers.org/so/SO:0000316':'CDS',
            'http://identifiers.org/so/SO:0000141':'Terminator',
            'http://identifiers.org/so/SO:0000704':'Gene',
            'http://identifiers.org/so/SO:0000057':'Operator',
            'http://identifiers.org/so/SO:0000280':'Engineered Gene',
            'http://identifiers.org/so/SO:0000234':'mRNA',
            'http://identifiers.org/chebi/CHEBI:35224':'Effector',
            'http://identifiers.org/go/GO:0003700':'Transcription Factor'}

# flag = False

# for cd in doc.componentDefinitions:
#     if len(cd.sequenceAnnotations) is not 0:
#         for sa in cd.sequenceAnnotations:
#             if sa.component is None:
#                 flag = True

copyCDs = []

for cd in doc.componentDefinitions:
    num = 0
    for sa in cd.sequenceAnnotations:
        if sa.name == 'misc_feature':
            sa.name = 'misc_feature' + str(num)
            num = num + 1
            
    copyCDs.append(cd)

for cd in copyCDs:
    if len(cd.components) == 0 and len(cd.sequenceAnnotations) != 0:
        for sa in cd.sequenceAnnotations:
            if sa.component == None:
                sa.extract()
                
# if len(doc.componentDefinitions) is 1:
#     cd = doc.componentDefinitions[0]
#     if len(cd.sequenceAnnotations) is not 0:
#         for sa in cd.sequenceAnnotations:
#             if sa.component is None:
#                 sa.extract()

numCD = len(doc.componentDefinitions)
contents = []
for i in range(numCD):
    contents.append(widgets.VBox())
    
accordion = widgets.Accordion(children = contents)
            
            
# Prints all ComponenentDefinitions in the SBOL file
for index, cd in enumerate(doc.componentDefinitions):
    box = contents[index]
    accordion.set_title(index, cd.displayId)
    childrenList = []
    htmlString = ''
    
    if cd.description is not None:
        htmlString = htmlString + '<dt><u>Description</u>:</dt>' + cd.description + '</dt>'
    if len(cd.types) is not 0:
        htmlString = htmlString + '<dt><u>Type(s)</u>:</dt>'
        for typeComponent in cd.types:
            if typeComponent in typeDict:
                htmlString = htmlString + '<dd>-&nbsp' + typeDict[typeComponent] + '</dd>'
            else:
                htmlString = htmlString + '<dd>-&nbspOther</dd>' 
    if len(cd.roles) is not 0:
        htmlString = htmlString + '<dt><u>Role(s)</u>:</dt>'
        for role in cd.roles:
            if role in roleDict:
                htmlString = htmlString + '<dd>-&nbsp' + roleDict[role] + '</dd>'
            else:
                htmlString = htmlString + '<dd>-&nbspOther</dd>'
    if len(cd.components) is not 0:
        htmlString = htmlString + '<dt><u>Components</u>:</dt>'
        for component in cd.components:
            htmlString = htmlString + '<dd>-&nbsp' + component.name + '</dd>'
            
    childrenList.append(widgets.HTML('<d1>' + htmlString + '</d1>'))
    box.children = childrenList

display(accordion)

Accordion(children=(VBox(children=(HTML(value='<d1><dt><u>Description</u>:</dt>araC-Pbad - Arabinose inducible…

# Adding Interactions
Select more than one component with `shift + click` or `control (cmd) + click`.

In [531]:
from sbol import *
import ipywidgets as widgets
from IPython.display import clear_output, Markdown

setHomespace('https://bu.edu/ben')

# modules = []

componentNames = []
mapComponents = {}

for cd in doc.componentDefinitions:
    componentNames.append(cd.displayId)
    mapComponents[cd.displayId] = cd
    
typesInteractions = ['Inhibition', 'Stimulation', 'Biochemical Reaction', 
                     'Non-Covalent Binding', 'Degradation', 'Genetic Production', 
                     'Control']
mapInteractions = {'Inhibition':'http://identifiers.org/biomodels.sbo/SBO:0000169', 
                   'Stimulation':'http://identifiers.org/biomodels.sbo/SBO:0000170', 
                   'Biochemical Reaction':'http://identifiers.org/biomodels.sbo/SBO:0000176', 
                     'Non-Covalent Binding':'http://identifiers.org/biomodels.sbo/SBO:0000177', 
                   'Degradation':'http://identifiers.org/biomodels.sbo/SBO:0000179', 
                   'Genetic Production':'http://identifiers.org/biomodels.sbo/SBO:0000589', 
                     'Control':'http://identifiers.org/biomodels.sbo/SBO:0000168'}

typesParticipants = ['Inhibitor', 'Inhibited', 'Stimulator', 'Stimulated', 'Reactant',
                     'Product', 'Promoter', 'Modifier', 'Modified', 'Template']
mapParticipants = {'Inhibitor':'http://identifiers.org/biomodels.sbo/SBO:0000020',
                   'Inhibited':'http://identifiers.org/biomodels.sbo/SBO:0000642',
                   'Stimulator':'http://identifiers.org/biomodels.sbo/SBO:0000459',
                   'Stimulated':'http://identifiers.org/biomodels.sbo/SBO:0000643',
                   'Reactant':'http://identifiers.org/biomodels.sbo/SBO:0000010',
                   'Product':'http://identifiers.org/biomodels.sbo/SBO:0000011',
                   'Promoter':'http://identifiers.org/biomodels.sbo/SBO:0000598',
                   'Modifier':'http://identifiers.org/biomodels.sbo/SBO:0000019',
                   'Modified':'http://identifiers.org/biomodels.sbo/SBO:0000644',
                   'Template':'http://identifiers.org/biomodels.sbo/SBO:0000645'}


participants = widgets.SelectMultiple(
    options = componentNames,
    description = 'Components:',
    rows = len(componentNames),
    disabled = False
)

interaction = widgets.Dropdown(
    options = typesInteractions,
    description = 'Interaction:',
    value = None,
    disabled = False
)

submitButton = widgets.Button(
    description = 'Submit',
    disabled = False,
    button_style = '',
    tooltip = 'Converts ',######## ADJKLAKLSDJL
    icon = ''
)

changeButton = widgets.Button(
    description = 'Change',
    disabled = False,
    button_style = '',
    tooltip = 'Changes interaction',
    icon = ''
)

createInteraction = widgets.Button(
    description = 'Create Interaction',
    disabled = False,
    button_style = '',
    tooltip = 'Creates interaction',
    icon = ''
)

interactionWidgets = widgets.VBox()

outputBox = widgets.Output()

display(participants)
display(interaction)
display(submitButton)
display(outputBox)

@outputBox.capture()
def printMessage(line):
    print(line)

    

module = ModuleDefinition('md0')
foundModule = None
for md in doc.moduleDefinitions:
    if module.identity == md.identity:
        foundModule = md

if foundModule is None:
    doc.addModuleDefinition(module)
else:
    module = foundModule


createdFCs = []
createdInteractions = []
createdMapsTos = []
    
def submitResponder(sender):
    global module, currentInteraction, originalCDs, createdMapsTos
    
    outputBox.clear_output()
    
    if len(participants.value) == 0:
        printMessage('You need to select components!')
        return
    
    items = []
    
    interactionString = interaction.value.replace(' ', '_').replace('-', '_')
    
    for p in participants.value:
        fc = None
        try:
            fc = module.functionalComponents.create(p)
            fc.definition = mapComponents[p].identity
            fc.access = SBOL_ACCESS_PUBLIC
            fc.direction = SBOL_DIRECTION_NONE
            
            createdFCs.append(fc.identity)
        except:
            pass
            
        items.append(widgets.Dropdown(options = typesParticipants, description = p + ':', disabled = False, 
                     style = {'description_width': 'initial'}))
        
        interactionString += '_' + p
        
        
        # Checking to see if parent component exists as a functional component
        for original in originalCDs:
            for c in original.components:
                if c.definition == mapComponents[p].identity:
                    fcP = None
                    try:
                        fcP = module.functionalComponents.create(original.displayId)
                        fcP.definition = original.identity
                        fcP.access = SBOL_ACCESS_PUBLIC
                        fcP.direction = SBOL_DIRECTION_NONE
                    except:
                        for fc in module.functionalComponents:
                            if fc.definition == original.identity:
                                fcP = fc
                                
                    if fc is not None:
                        if fc.definition == c.definition:
                            foundMap = False
                            for mt in fcP.mapsTos:
                                if mt.displayId == str('map_' + fc.displayId):
                                    foundMap = True
                            
                            if foundMap == False:
                                mapsto = fcP.mapsTos.create('map_' + fc.displayId)
                        
                                mapsto.refinement = SBOL_REFINEMENT_USE_LOCAL
                                mapsto.local = fc.identity
                                mapsto.remote = c.identity
                                
                                createdMapsTos.append(mapsto.identity)
    
    
    interactionWidgets.children = items
    
    
    interactionModule = module.interactions.create(interactionString)
    interactionModule.types = mapInteractions[interaction.value]
    currentInteraction = interactionModule
    createdInteractions.append(interactionModule.identity)
    
    
    participants.disabled = True
    interaction.disabled = True
    submitButton.disabled = True
    
    if changeButton.layout.display == 'none':
        changeButton.layout.display = None
        interactionWidgets.layout.display = None
        createInteraction.layout.display = None
    else:
        display(changeButton)
        display(interactionWidgets)
        display(createInteraction)
    


def changeResponder(sender):
    global createdFCs, createdInteractions, createdMapsTos
    
    interactionWidgets.children = []
    interactionWidgets.hidden = True
    interactionWidgets.layout.display = 'none'
    changeButton.layout.display = 'none'
    createInteraction.layout.display = 'none'
    
    participants.disabled = False
    interaction.disabled = False
    submitButton.disabled = False
    
    for mapsTosURI in createdMapsTos:
        for fc in module.functionalComponents:
            exists = False
            for mt in fc.mapsTos:
                if mapsTosURI == mt.identity:
                    exists = True
            if exists == True:
                fc.mapsTos.remove(mapsTosURI)
    
    for fcURI in createdFCs:
        module.functionalComponents.remove(fcURI)
        
    for iURI in createdInteractions:
        module.interactions.remove(iURI)
        
    createdFCs = []
    createdInteractions = []
    createdMapsTos = []
    
    
def createResponder(sender):
    global createdFCs, createdInteractions, createdMapsTos, module
    alreadyExist = False
    
    
    for index, widget in enumerate(interactionWidgets.children):
        participation = currentInteraction.participations.create(widget.description[:-1])
        participation.roles = mapParticipants[widget.value]
        
        fcIdentity = ''
        for fc in module.functionalComponents:
            if fc.displayId == widget.description[:-1]:
                fcIdentity = fc.identity
        
        participation.participant = fcIdentity
    

    
    interactionWidgets.children = []
    interactionWidgets.hidden = True
    interactionWidgets.layout.display = 'none'
    changeButton.layout.display = 'none'
    createInteraction.layout.display = 'none'
    
    participants.disabled = False
    interaction.disabled = False
    submitButton.disabled = False
    
    participants.value = []
    interaction.value = None
    
    createdFCs = []
    createdInteractions = []
    createdMapsTos = []
    
    if alreadyExist == False:
        printMessage('Interaction created!')
        
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))



submitButton.on_click(submitResponder)
changeButton.on_click(changeResponder)
createInteraction.on_click(createResponder)

SelectMultiple(description='Components:', options=('BBa_K808000', 'misc_feature0', 'misc_feature1', 'CDS', 'pr…

Dropdown(description='Interaction:', options=('Inhibition', 'Stimulation', 'Biochemical Reaction', 'Non-Covale…

Button(description='Submit', style=ButtonStyle(), tooltip='Converts ')

Output()

Button(description='Change', style=ButtonStyle(), tooltip='Changes interaction')

Button(description='Create Interaction', style=ButtonStyle(), tooltip='Creates interaction')

<IPython.core.display.Javascript object>

# Interactions

In [532]:
from sbol import *
import ipywidgets as widgets
from IPython.display import clear_output, Markdown

typesInteractions = ['Inhibition', 'Stimulation', 'Biochemical Reaction', 
                     'Non-Covalent Binding', 'Degradation', 'Genetic Production', 
                     'Control']
interactionMap = {'http://identifiers.org/biomodels.sbo/SBO:0000169':'Inhibition', 
                   'http://identifiers.org/biomodels.sbo/SBO:0000170':'Stimulation', 
                   'http://identifiers.org/biomodels.sbo/SBO:0000176':'Biochemical Reaction', 
                   'http://identifiers.org/biomodels.sbo/SBO:0000177':'Non-Covalent Binding', 
                   'http://identifiers.org/biomodels.sbo/SBO:0000179':'Degradation', 
                   'http://identifiers.org/biomodels.sbo/SBO:0000589':'Genetic Production', 
                   'http://identifiers.org/biomodels.sbo/SBO:0000168':'Control'}

typesParticipants = ['Inhibitor', 'Inhibited', 'Stimulator', 'Stimulated', 'Reactant',
                     'Product', 'Promoter', 'Modifier', 'Modified', 'Template']
participantMap = {'http://identifiers.org/biomodels.sbo/SBO:0000020':'Inhibitor',
                   'http://identifiers.org/biomodels.sbo/SBO:0000642':'Inhibited',
                   'http://identifiers.org/biomodels.sbo/SBO:0000459':'Stimulator',
                   'http://identifiers.org/biomodels.sbo/SBO:0000643':'Stimulated',
                   'http://identifiers.org/biomodels.sbo/SBO:0000010':'Reactant',
                   'http://identifiers.org/biomodels.sbo/SBO:0000011':'Product',
                   'http://identifiers.org/biomodels.sbo/SBO:0000598':'Promoter',
                   'http://identifiers.org/biomodels.sbo/SBO:0000019':'Modifier',
                   'http://identifiers.org/biomodels.sbo/SBO:0000644':'Modified',
                   'http://identifiers.org/biomodels.sbo/SBO:0000645':'Template'}

interactions = doc.moduleDefinitions[0].interactions
numInteractions = len(interactions)
contents = []
for i in range(numInteractions):
    contents.append(widgets.VBox())
    
accordion = widgets.Accordion(children = contents)

for index, inter in enumerate(interactions):
    box = contents[index]
    accordion.set_title(index, inter.displayId)
    childrenList = []
    htmlString = ''
    
    htmlString += '<dt><u>Interaction Type(s)</u>:</dt>'
    for typeI in inter.types:
        htmlString += '<dd>' + interactionMap[typeI] + '</dd>'
    
    htmlString += '<dt><u>Participants</u>:</dt>'
    for participation in inter.participations:
        htmlString += '<dd>-&nbsp' + participation.displayId
        for role in participation.roles:
            htmlString += str('&nbsp(' + participantMap[role] + ')')
        htmlString += '</dd>'

    childrenList.append(widgets.HTML('<d1>' + htmlString + '</d1>'))
    box.children = childrenList
    

display(accordion)



Accordion(children=(VBox(children=(HTML(value='<d1><dt><u>Interaction Type(s)</u>:</dt><dd>Genetic Production<…

# Temporary Save Button

In [533]:
from sbol import *
import ipywidgets as widgets
from IPython.display import clear_output, Markdown

saveButton = widgets.Button(
    description = 'Save',
    disabled = False,
    button_style = '',
    tooltip = 'Saves file',
    icon = ''
)

display(saveButton)

def saveStuff(sender):
    doc.write('hello.xml')
    
saveButton.on_click(saveStuff)


Button(description='Save', style=ButtonStyle(), tooltip='Saves file')

# Upload to SynBioHub
To upload to SynBioHub:
1. Enter your username and password (you need to have an account to upload).
2. Enter a name, description, and version for the Collection.
3. Press Submit.

In [577]:
from sbol import *
import ipywidgets as widgets
from IPython.display import clear_output, Markdown

loginLabel = widgets.HTML('<font size="5"><b>Login to SynBioHub</b></font>')

usernameField = widgets.componentName = widgets.Text(
    value = '',
    placeholder = 'Enter email',
    description = 'Email:',
    disabled = False,
)

passwordField = widgets.Password(description = 'Password:', placeholder = 'Enter password')

loginSubmitButton = widgets.Button(
    description = 'Submit',
    disabled = False,
    button_style = '',
    tooltip = 'Submit annotated Document',
    icon = ''
)

errorBox = widgets.Output()

# docDisplayId = widgets.componentName = widgets.Text(
#     value = '',
#     placeholder = 'Enter display id',
#     description = 'Username:',
#     disabled = False,
# )

# Name = Display Id
propertiesLabel = widgets.HTML('<font size="5"><b>Enter Name, Description, and Version</b></font>')

docName = widgets.componentName = widgets.Text(
    value = '',
    placeholder = 'Enter name',
    description = 'Name:',
    disabled = False,
)

docDescription = widgets.componentName = widgets.Text(
    value = '',
    placeholder = 'Enter description',
    description = 'Description:',
    disabled = False,
)

docVersion = widgets.componentName = widgets.Text(
    value = '',
    placeholder = 'Enter version',
    description = 'Version:',
    disabled = False,
)

display(loginLabel)
display(usernameField)
display(passwordField)
display(propertiesLabel)
display(docName)
display(docDescription)
display(docVersion)
display(loginSubmitButton)
display(errorBox)

sbh = PartShop('https://synbiohub.org')



@errorBox.capture()
def loginSBH(sender):
    clear_output()
    if usernameField.value == '':
        print('You need a username!')
        return
    if passwordField.value == '':
        print('You need a password!')
        return
    if docName.value == '':
        print('You need a name for the Collection!')
        return
    if docDescription.value == '':
        print('You need a description for the Collection!')
        return
    if docVersion.value == '':
        print('You need a version for this Collection!')
        return
    
    sbh.login(usernameField.value, passwordField.value)
    
    doc.displayId = docName.value
    doc.name = docName.value
    doc.description = docDescription.value
    doc.version = docVersion.value

    message = sbh.submit(doc)
    cleanedMessage = message.replace('[','').replace('\"','').replace(']','')
    print(cleanedMessage)


loginSubmitButton.on_click(loginSBH)





HTML(value='<font size="5"><b>Login to SynBioHub</b></font>')

Text(value='', description='Email:', placeholder='Enter email')

Password(description='Password:', placeholder='Enter password')

HTML(value='<font size="5"><b>Enter Name, Description, and Version</b></font>')

Text(value='', description='Name:', placeholder='Enter name')

Text(value='', description='Description:', placeholder='Enter description')

Text(value='', description='Version:', placeholder='Enter version')

Button(description='Submit', style=ButtonStyle(), tooltip='Submit annotated Document')

Output()

In [559]:
doc.displayId
doc.name

'testing'

In [541]:
for md in doc.moduleDefinitions:
    for fc in md.functionalComponents:
        for i in fc.mapsTos:
            print(i)

https://bu.edu/ben/md0/BBa_K808000/map_CDS/1.0.0
https://bu.edu/ben/md0/BBa_K808000/map_promoter/1.0.0
